# Boundary vector score (BVS) - datajoint integration

Datajoint integration of bvs calculation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Set up basics
import sys, os
import numpy as np 
from collections import OrderedDict
# Make plots pretty 
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style='white')
%config InlineBackend.figure_format = 'retina'

import datajoint as dj

In [3]:
# Load base schema
schema = dj.schema(dj.config['dj_imaging.database'])
schema.spawn_missing_classes()

Connecting horsto@kavlidatajoint02.it.ntnu.no:3306


### Schema components

In [4]:
# Load personal schema 
borderscore_schema = dj.schema('user_horsto_borderscore')
borderscore_schema.spawn_missing_classes()

In [62]:
borderscore_schema.jobs

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__b_v_score,41ca617ce4c2db5edd7c09f5a57d323f,error,=BLOB=,"AssertionError: Please make sure ""fieldmap"" is a 2D array where field coordinates = 1 and rest = 0",=BLOB=,horsto@10.212.25.16,jupyterstack,3065,1066,2020-08-30 20:19:16
__shuffled_b_v_s,0646c99294862e131b74e1e1e389ac32,reserved,=BLOB=,,=BLOB=,horsto@10.212.25.16,jupyterstack,6278,1114,2020-08-30 20:47:16
__shuffled_b_v_s,06cf20253986c9379ac17f33f2cfec57,reserved,=BLOB=,,=BLOB=,horsto@10.212.25.16,jupyterstack,6600,1116,2020-08-30 20:47:17
__shuffled_b_v_s,0e66bcc36d47625b6b26fe73ad84cd92,reserved,=BLOB=,,=BLOB=,horsto@10.212.25.16,jupyterstack,9358,1133,2020-08-30 20:45:34
__shuffled_b_v_s,1065f540d645f521c9caa477b6a0a404,reserved,=BLOB=,,=BLOB=,horsto@10.212.25.16,jupyterstack,7742,1123,2020-08-30 20:47:18
__shuffled_b_v_s,37e21a8419ffa551e49cca279733f823,reserved,=BLOB=,,=BLOB=,horsto@10.212.25.16,jupyterstack,8871,1130,2020-08-30 20:45:32
__shuffled_b_v_s,3be3cf2aee2e6ee8cf2abe434f147e95,reserved,=BLOB=,,=BLOB=,horsto@10.212.25.16,jupyterstack,5952,1112,2020-08-30 20:45:13
__shuffled_b_v_s,407f86aba9c743d45157499248106719,reserved,=BLOB=,,=BLOB=,horsto@10.212.25.16,jupyterstack,6442,1115,2020-08-30 20:45:52
__shuffled_b_v_s,42da40a22708c488dd8a0f2943d73e22,reserved,=BLOB=,,=BLOB=,horsto@10.212.25.16,jupyterstack,5792,1111,2020-08-30 20:46:25
__shuffled_b_v_s,444da9c8e072822a2779e655270f9ced,reserved,=BLOB=,,=BLOB=,horsto@10.212.25.16,jupyterstack,8226,1126,2020-08-30 20:47:05


In [58]:
BVField().progress()

BVField              Completed 180392 of 180392 (100.0%)   2020-08-30 20:37:15


(0, 180392)

In [57]:
BVScore().progress()

BVScore              Completed 360783 of 360784 (100.0%)   2020-08-30 20:37:12


(1, 360784)

### Shuffling

In [8]:
ShuffleParams()

"shuffle_params_id Parameter set ID, starting with A",number_shuffles Expected number of shuffles,"margin_seconds Margin in seconds at start, end of sync samples",break_seconds Break in seconds between sync samples (-> non-continuous sampling of sync stream)
A,500,2.0,0.5


In [10]:
from dj_schemas.shuffling_bvs import * 

In [67]:
ShuffledBVS.progress()

ShuffledBVS          Completed 439 of 360784 (0.1%)   2020-08-30 20:54:58


(360345, 360784)

In [66]:
ShuffledBVS()

metasession_name Meta session name (hash),session_order Order of session within meta sessions (zero index!),"session_name Session name: Hash of animal_id, datasource_id, timestamp and combined 'yes'/'no' label",signal_dataset 16 character hash,cell_id Cell ID,channel the channel that this trace comes from (ROI masks are always primary),"spike_filter_id Parameter set ID, starting with A",tracking_dataset 16 character hash,"trackingparams_id Parameter set ID, starting with A","shuffle_params_id Parameter set ID, starting with A","s_t_params_id Parameter set ID, starting with A",signal_type Signal type (Fluorescence vs. Spikes),"map_params_id Parameter set ID, starting with A","field_params_id Parameter set ID, starting with A","bvfield_params_id Parameter set ID, starting with A",bv_field_dect_method Specifies how fields were extracted,number_shuffles Total number of shuffles (can vary from expected number),shuffling_offsets Shuffling offsets
02b50b09ffe61c44,0,84f50041aa7d1c2d,6159e2637dca1f7a,38,primary,A,1e50c374d80e56c7,A,A,A,df_f,B,A,A,opexebo,500,=BLOB=
02b50b09ffe61c44,0,84f50041aa7d1c2d,6159e2637dca1f7a,103,primary,A,1e50c374d80e56c7,A,A,A,df_f,A,A,A,bvs,500,=BLOB=
02c2eb88e2ca1252,0,348c22031e5e1e21,8e3f1636819358a8,16,primary,A,fae596f251472b4c,A,A,A,spikes,B,A,A,bvs,500,=BLOB=
04304fa38f6568fb,1,84e7e34eb08b20cd,924b5fdb54a57bb1,2,primary,A,9802409d8bd12be3,A,A,A,spikes,A,A,A,bvs,500,=BLOB=
05304a7b5f0d845a,0,5b92b96313c3fc19,65f42354d0827ba1,127,primary,A,b9a87ac17e7b5ad6,A,A,A,df_f,B,A,A,bvs,500,=BLOB=
05304a7b5f0d845a,0,5b92b96313c3fc19,65f42354d0827ba1,208,primary,A,b9a87ac17e7b5ad6,A,A,A,spikes,B,A,A,opexebo,500,=BLOB=
05304a7b5f0d845a,0,5b92b96313c3fc19,65f42354d0827ba1,231,primary,A,b9a87ac17e7b5ad6,A,A,A,spikes,A,A,A,bvs,500,=BLOB=
088eaba999b6c42f,0,dbc7e204d3b73727,0481a760f6e94378,54,primary,A,6aa03f8aadbd7920,A,A,A,df_f,A,A,A,opexebo,500,=BLOB=
088eaba999b6c42f,1,ef846f5b97b58189,0481a760f6e94378,47,primary,A,aab473c25308bf82,A,A,A,spikes,A,A,A,opexebo,500,=BLOB=
088eaba999b6c42f,2,78bc7cdcdf4d237e,0481a760f6e94378,69,primary,A,92244dd52cacb6ca,A,A,A,spikes,A,A,A,bvs,500,=BLOB=
